In [1]:
import torch.nn as nn
from torchmetrics.functional import dice
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import glob
from IPython.display import display
import os

import torch
import torch.nn as nn
import torch.nn.functional as F
from torchvision import datasets, transforms, models
from torch.utils.data import DataLoader
from torch_train import TorchTrain

In [6]:
BASE_DIR = "/kaggle/input/google-research-identify-contrails-reduce-global-warming"
DEVICE = TorchTrain.DEVICE

In [7]:
metadata = pd.read_csv("metadata.csv")
train_md = metadata[metadata["Split"] == "train"]
validation_md = metadata[metadata["Split"] == "validation"]
display(train_md.head())
display(validation_md.head())

,Record_ID,Split,Contrails,High_Pixels
0,1000216489776414077,train,False,0.000000
1,1000603527582775543,train,True,8.512878
2,1000660467359258186,train,True,1.028442
3,100071707854144929,train,False,0.000000
4,1000823728928031783,train,True,0.563049


,Record_ID,Split,Contrails,High_Pixels
20529,1000834164244036115,validation,False,0.000000
20530,1002653297254493116,validation,False,0.000000
20531,1002777035567823518,validation,True,0.038147
20532,1010397530434035516,validation,False,0.000000
20533,1012978360687713914,validation,False,0.000000


In [8]:
_T11_BOUNDS = (243, 303)
_CLOUD_TOP_TDIFF_BOUNDS = (-4, 5)
_TDIFF_BOUNDS = (-4, 2)
N_TIMES_BEFORE = 4
IMG_SIZE = 256
BATCH_SIZE = 16

def normalize_range(data, bounds):
    """Maps data to the range [0, 1]."""
    return (data - bounds[0]) / (bounds[1] - bounds[0])

def load_one_record(record_id, BASE_DIR=BASE_DIR, mask_too=True):
    band11 = np.load(f"{BASE_DIR}/{str(record_id)}/band_11.npy")
    band14 = np.load(f"{BASE_DIR}/{str(record_id)}/band_14.npy")
    band15 = np.load(f"{BASE_DIR}/{str(record_id)}/band_15.npy")
    if mask_too:
        human_pixel_mask =  np.load(f"{BASE_DIR}/{str(record_id)}/human_pixel_masks.npy")
        
    r = normalize_range(band15 - band14, _TDIFF_BOUNDS)
    g = normalize_range(band14 - band11, _CLOUD_TOP_TDIFF_BOUNDS)
    b = normalize_range(band14, _T11_BOUNDS)
    false_color = np.clip(np.stack([r, g, b], axis=2), 0, 1)
    
    img = false_color[..., N_TIMES_BEFORE]
    if mask_too:
        return img, human_pixel_mask
    else:
        return img

In [16]:
idx = 10

In [21]:
record_id = train_md.iloc[idx]["Record_ID"]
contrails_presnt = train_md.iloc[idx]["Contrails"]
contrails_presnt = torch.tensor(contrails_presnt).to(torch.float32)
images, mask = torch.ones((256, 256, 3)), torch.ones((256, 256, 1))
# images = torch.tensor(np.reshape(images, (256, 256, 3))).to(torch.float32).permute(2, 0, 1)
# images = self.normalize_image(images)
images = images.transpose(2, 0, 1)
mask = mask.transpose(2, 0, 1)
outputs = torch.concatenate((mask, contrails_presnt.unsqueeze(0)), dim=0)
images.float(), outputs

/tmp/ipykernel_22586/4247539866.py:3: DeprecationWarning: In future, it will be an error for 'np.bool_' scalars to be interpreted as an index
  contrails_presnt = torch.tensor(contrails_presnt).to(torch.float32)


TypeError: transpose() received an invalid combination of arguments - got (int, int, int), but expected one of:
 * (int dim0, int dim1)
 * (name dim0, name dim1)


In [37]:
tensor1 = torch.randn(3, 4)
tensor2 = torch.randn(2, 2)

# Concatenate the two tensors along the first dimension
tensor = torch.cat((tensor1, tensor2), dim=0)

print(tensor)

RuntimeError: Sizes of tensors must match except in dimension 0. Expected size 4 but got size 2 for tensor number 1 in the list.

In [40]:
mask = np.ones((256, 256, 1))
classes = np.ones((1, 1))
mask = torch.tensor(mask)
classes = torch.tensor(classes)
outputs = (mask, classes)

In [43]:
outputs.to("cpu")

AttributeError: 'tuple' object has no attribute 'to'

In [42]:
# mask = np.ones((256, 256, 1))
# classes = [[1]]
# outputs = [mask, classes]
# output = torch.tensor([mask, classes])
# outputs

In [32]:
a, b = outputs

In [11]:
class ContrailsDataLoader():
    def __init__(self, df, split="train"):
        self.df = df
        self.split = split
        self.normalize_image = transforms.Normalize((0.485, 0.456, 0.406), (0.229, 0.224, 0.225))
        
    def __getitem__(self, idx):
        record_id = self.df.iloc[idx]["Record_ID"]
        contrails_presnt = self.df.iloc[idx]["Contrails"]
        contrails_presnt = torch.tensor(contrails_presnt).to(torch.float32)
        images, mask = load_one_record(record_id, f"{BASE_DIR}/{self.split}")
        images = torch.tensor(np.reshape(images, (256, 256, 3))).to(torch.float32).permute(2, 0, 1)
        images = self.normalize_image(images)
        mask = mask.transpose(2, 0, 1)
        outputs = torch.concatenate((mask, contrails_presnt.unsqueeze(0)), dim=0)
        return images.float(), outputs

    def __len__(self):
        return len(self.df)
    
class ContrailsDataLoaderTest():
    def __init__(self, ids, split="test"):
        self.ids = ids
        self.split = split
        self.normalize_image = transforms.Normalize((0.485, 0.456, 0.406), (0.229, 0.224, 0.225))
        
    def __getitem__(self, idx):
        record_id = self.ids[idx]
        images = load_one_record(record_id, f"{BASE_DIR}/{self.split}", mask_too=False)
        images = torch.tensor(np.reshape(images, (256, 256, 3))).to(torch.float32).permute(2, 0, 1)
        images = self.normalize_image(images)
        return images.float(), record_id

    def __len__(self):
        return len(self.ids)

In [12]:
train_dataloader = ContrailsDataLoader(train_md, "train")
validation_dataloader = ContrailsDataLoader(validation_md, "validation")

train_data = DataLoader(train_dataloader, batch_size=BATCH_SIZE, shuffle=True, num_workers=2)
validation_data = DataLoader(validation_dataloader, batch_size=BATCH_SIZE, shuffle=True, num_workers=2)


In [13]:
class BCEWithLogitsLossUpdated:
    def __init__(self, weight=None, size_average=None, reduce=None, reduction='mean', pos_weight=None):
        self.weight = weight
        self.size_average = size_average
        self.reduce = reduce
        self.reduction = reduction
        self.pos_weight = pos_weight

    def __call__(self, inputs, targets):
        mask_pred, class_pred = inputs
        mask_true, class_true = targets
        mask_loss = F.binary_cross_entropy_with_logits(mask_pred, mask_true, self.weight, self.size_average, self.reduce, self.reduction, self.pos_weight)
        class_loss = F.binary_cross_entropy_with_logits(class_pred, class_true, self.weight, self.size_average, self.reduce, self.reduction, self.pos_weight)
        return mask_loss + class_loss